# Quasi-Geostrophic Omega Equation (Traditional)

In [ ]:
from datetime import datetime

import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cartopy.util as cutil
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import metpy.calc as mpcalc
import metpy.constants as mpconstants
from metpy.units import units
import numpy as np
from scipy.ndimage import gaussian_filter
import xarray as xr

In [ ]:
date = datetime(2019, 3, 14, 12)

# Current Data
# ds = xr.open_dataset('https://thredds.ucar.edu/thredds/dodsC/grib/NCEP/GFS/Global_0p5deg_ana/TP')

# UCAR RDA Archive GFS Files 1 deg output
# ds = xr.open_dataset('https://rda.ucar.edu/thredds/dodsC/aggregations/g/ds083.2/2/TP')

# UCAR RDA Archive - Global 0.25 deg Data
# ds = xr.open_dataset('https://rda.ucar.edu/thredds/dodsC/files/g/ds083.3/'
#                      f'{date:%Y}/{date:%Y%m}/gdas1.fnl0p25.{date:%Y%m%d%H}.f00.grib2')

# NCEI Data Archive
if date > datetime(2020, 5, 15, 6):
    ds = xr.open_dataset(f'https://www.ncei.noaa.gov/thredds/dodsC/model-gfs-g4-anl-files/'
                         f'{date:%Y%m}/{date:%Y%m%d}/gfs_4_{date:%Y%m%d_%H}00_000.grb2')
else:
    #'https://www.ncei.noaa.gov/thredds/dodsC/model-gfs-g4-anl-files-old/202005/20200515/gfsanl_4_20200515_0000_000.grb2'
    ds = xr.open_dataset(f'https://www.ncei.noaa.gov/thredds/dodsC/model-gfs-g4-anl-files-old/'
                         f'{date:%Y%m}/{date:%Y%m%d}/gfsanl_4_{date:%Y%m%d_%H}00_000.grb2')

# Local File
#ds = xr.open_dataset('groundhogs_day_blizzard/GFS_{0:%Y%m%d}_{0:%H}00.nc'.format(date))

lats = ds.lat.sel(lat=slice(60, 10)).values
lons = ds.lon.sel(lon=slice(360-160, 360-50)).values

subset = dict(vertical=700 * units.hPa, time=date, lat=slice(60, 10), lon=slice(360-160, 360-50))
hght_700 = ds['Geopotential_height_isobaric'].metpy.sel(subset).metpy.quantify()
tmpk_700 = ds['Temperature_isobaric'].metpy.sel(subset).metpy.quantify()
uwnd_700 = ds['u-component_of_wind_isobaric'].metpy.sel(subset).metpy.quantify()
vwnd_700 = ds['v-component_of_wind_isobaric'].metpy.sel(subset).metpy.quantify()
subset['vertical'] = 500 * units.hPa
uwnd_500 = ds['u-component_of_wind_isobaric'].metpy.sel(subset).metpy.quantify()
vwnd_500 = ds['v-component_of_wind_isobaric'].metpy.sel(subset).metpy.quantify()
subset['vertical'] = 900 * units.hPa
uwnd_900 = ds['u-component_of_wind_isobaric'].metpy.sel(subset).metpy.quantify()
vwnd_900 = ds['v-component_of_wind_isobaric'].metpy.sel(subset).metpy.quantify()

ntime = ds.Geopotential_height_isobaric.dims[0]
vtime = hght_700[ntime].values.astype('datetime64[ms]').astype('O')

# For 1 deg data use 20
# For 0.5/0.25 deg data use 80
smoothing_passes = 80

hght_700s = mpcalc.smooth_n_point(hght_700, 9, smoothing_passes)

tmpk_700s = mpcalc.smooth_n_point(tmpk_700, 9, smoothing_passes)
uwnd_700s = mpcalc.smooth_n_point(uwnd_700, 9, smoothing_passes)
vwnd_700s = mpcalc.smooth_n_point(vwnd_700, 9, smoothing_passes)
uwnd_500s = mpcalc.smooth_n_point(uwnd_500, 9, smoothing_passes)
vwnd_500s = mpcalc.smooth_n_point(vwnd_500, 9, smoothing_passes)
uwnd_900s = mpcalc.smooth_n_point(uwnd_900, 9, smoothing_passes)
vwnd_900s = mpcalc.smooth_n_point(vwnd_900, 9, smoothing_passes)
    
tmpc_700 = tmpk_700s.metpy.convert_units('degC')

sigma = 2.0e-6 * units('m^2 Pa^-2 s^-2')
f0 = 1e-4 * units('s^-1')
Rd = mpconstants.Rd

avor_900 = mpcalc.absolute_vorticity(uwnd_900s, vwnd_900s)
avor_500 = mpcalc.absolute_vorticity(uwnd_500s, vwnd_500s)

vortadv_900 = mpcalc.advection(avor_900, uwnd_900s, vwnd_900s)
vortadv_500 = mpcalc.advection(avor_500, uwnd_500s, vwnd_500s)

diff_avor = ((vortadv_900.metpy.unit_array - vortadv_500.metpy.unit_array).to_base_units()/(400 * units.hPa))

term_A = (-f0/sigma*diff_avor).to_base_units()

tadv_700 = mpcalc.advection(tmpk_700s, uwnd_700s, vwnd_700s)
lap_tadv_700 = mpcalc.laplacian(tadv_700)


term_B = (-Rd/(sigma*(700 * units.hPa))*lap_tadv_700.metpy.unit_array).to_base_units()

In [ ]:
# Set range for color-filled forcing terms
clevs_omega = np.arange(-10, 10.5, 1)

mapcrs = ccrs.LambertConformal(central_longitude=-100, central_latitude=35, standard_parallels=(30, 60))
datacrs = ccrs.PlateCarree()

# Transform Coordinates ahead of time
clons, clats = np.meshgrid(lons, lats)
tlatlons = mapcrs.transform_points(ccrs.PlateCarree(),clons,clats)
clons = tlatlons[:,:,0]
clats = tlatlons[:,:,1]

clevs_700_tmpc = np.arange(-40, 41, 2)
clevs_700_hght = np.arange(0, 8000, 30)

cmap = plt.cm.BrBG_r

# For 1 deg data use 4
# For 0.5 deg data use 5
# For 0.25 deg data use 8
barb_spacing = 8

wind_slice = (slice(None, None, barb_spacing), slice(None, None, barb_spacing))

# 1st image
fig = plt.figure(1, figsize=(17,15))
ax1 = plt.subplot(111, projection=mapcrs)
ax1.set_extent([-130, -72, 20, 55], ccrs.PlateCarree())
ax1.add_feature(cfeature.COASTLINE.with_scale('50m'))
ax1.add_feature(cfeature.STATES.with_scale('50m'))

cf = ax1.contourf(clons, clats, -term_A*1e12, clevs_omega, cmap=cmap, extend='both')
plt.colorbar(cf, orientation='horizontal', pad=0, aspect=50, extendrect=True)

csf = ax1.contour(clons, clats, tmpc_700, clevs_700_tmpc, colors='grey', linestyles='dashed')
plt.clabel(csf, fmt='%d')

cs = ax1.contour(clons, clats, hght_700s, clevs_700_hght, colors='black')
plt.clabel(cs, fmt='%d')

ax1.barbs(lons[wind_slice[0]], lats[wind_slice[0]],
          uwnd_700s[wind_slice].metpy.convert_units('kt').values,
          vwnd_700s[wind_slice].metpy.convert_units('kt').values,
          pivot='middle', color='black', transform=datacrs)

plt.title('700-hPa GFS Geo. HGHT (m), TMPC, Inverted QG-Omega diff. Abs. Vort. Adv. '
          '($*10^{12}$ kg m$^{-3}$ s$^{-3}$) and Wind Barbs (kt)', loc='left')
plt.title(f'Valid Time: {vtime}', loc='right')

plt.savefig(f'QGOmega_Vort_{date:%Y%m%d_%H}00.png', bbox_inches='tight')
plt.show()

In [ ]:
# 2nd image
fig = plt.figure(1, figsize=(17,15))
ax2 = plt.subplot(111, projection=mapcrs)
ax2.set_extent([-130, -72, 20, 55], ccrs.PlateCarree())
ax2.add_feature(cfeature.COASTLINE.with_scale('50m'))
ax2.add_feature(cfeature.STATES.with_scale('50m'))

cf = ax2.contourf(clons, clats, -term_B*1e12, clevs_omega, cmap=cmap, extend='both')
plt.colorbar(cf, orientation='horizontal', pad=0, aspect=50, extendrect=True)

csf = ax2.contour(clons, clats, tmpc_700, clevs_700_tmpc, colors='grey', linestyles='dashed')
plt.clabel(csf, fmt='%d')

cs = ax2.contour(clons, clats, hght_700s, clevs_700_hght, colors='black')
plt.clabel(cs, fmt='%d')

ax2.barbs(lons[wind_slice[0]], lats[wind_slice[0]],
          uwnd_700s[wind_slice].metpy.convert_units('kt').values,
          vwnd_700s[wind_slice].metpy.convert_units('kt').values,
          pivot='middle', color='black', transform=datacrs)

plt.title('700-hPa GFS Geo. HGHT (m), TMPC, Inverted QG-Omega Lap. Temp. Adv. '
          '($*10^{12}$ kg m$^{-3}$ s$^{-3}$), and Wind Barbs (kt)', loc='left')
plt.title(f'Valid Time: {vtime}', loc='right')

plt.savefig(f'QGOmega_Temp_{date:%Y%m%d_%H}00.png', bbox_inches='tight')
plt.show()

In [ ]:
# 3rd image
fig = plt.figure(1, figsize=(17,15))
ax3 = plt.subplot(111, projection=mapcrs)
ax3.set_extent([-130, -72, 20, 55], ccrs.PlateCarree())
ax3.add_feature(cfeature.COASTLINE.with_scale('50m'))
ax3.add_feature(cfeature.STATES.with_scale('50m'))

cf = ax3.contourf(clons, clats, -(term_A+term_B)*1e12, clevs_omega, cmap=cmap, extend='both')
plt.colorbar(cf, orientation='horizontal', pad=0, aspect=50, extendrect=True)

csf = ax3.contour(clons, clats, tmpc_700, clevs_700_tmpc, colors='grey', linestyles='dashed')
plt.clabel(csf, fmt='%d')

cs = ax3.contour(clons, clats, hght_700s, clevs_700_hght, colors='black')
plt.clabel(cs, fmt='%d')

ax3.barbs(lons[wind_slice[0]], lats[wind_slice[0]],
          uwnd_700s[wind_slice].metpy.convert_units('kt').values,
          vwnd_700s[wind_slice].metpy.convert_units('kt').values,
          pivot='middle', color='black', transform=datacrs)

plt.title('700-hPa GFS Geo. HGHT (m), TMPC, Inverted QG-Omega Trad. '
          '($*10^{12}$ kg m$^{-3}$ s$^{-3}$) and Wind Barbs (kt)', loc='left')
plt.title(f'Valid Time: {vtime}', loc='right')

plt.savefig(f'QGOmega_Total_{date:%Y%m%d_%H}00.png', bbox_inches='tight')
plt.show()